In [1]:
!nvidia-smi

Fri Dec 22 17:32:31 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   66C    P8              11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
!pip install -Uqqq pip --progress-bar off
!pip install -qqq bitsandbytes==0.40.0 --progress-bar off
!pip install -qqq torch==2.0.1 --progress-bar off
!pip install -qqq transformers==4.30.0 --progress-bar off
!pip install -qqq accelerate==0.21.0 --progress-bar off
!pip install -qqq xformers==0.0.20 --progress-bar off
!pip install -qqq einops==0.6.1 --progress-bar off
!pip install -qqq langchain==0.0.233 --progress-bar off
!pip install pyngrok

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.1.0+cu121 requires torch==2.1.0, but you have torch 2.0.1 which is incompatible.
torchdata 0.7.0 requires torch==2.1.0, but you have torch 2.0.1 which is incompatible.
torchtext 0.16.0 requires torch==2.1.0, but you have torch 2.0.1 which is incompatible.
torchvision 0.16.0+cu121 requires torch==2.1.0, but you have torch 2.0.1 which is incompatible.


In [37]:
import re
from pyngrok import ngrok,conf
import requests
import torch
import smtplib
from email.mime.text import MIMEText
import socket
from langchain import PromptTemplate
from langchain.chains import ConversationChain
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
from langchain.llms import HuggingFacePipeline
from langchain.schema import BaseOutputParser
from flask import Flask, render_template, request,jsonify
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    StoppingCriteria,
    StoppingCriteriaList,
    pipeline,
    Conversation,
    ConversationalPipeline
)
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore
cred = credentials.Certificate(r"/content/classsnapv2-firebase-adminsdk-vhajc-13a82f347f.json")
firebase_admin.initialize_app(cred)

db=firestore.client()

socket.getaddrinfo('localhost',8080)
# warnings.filterwarnings("ignore", category=UserWarning)

## Load Model

In [5]:
MODEL_NAME = "tiiuae/falcon-7b"#"rsvp-ai/bertserini-bert-base-squad"#
# !git clone https://huggingface.co/tiiuae/falcon-7b
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME, trust_remote_code=True, load_in_8bit=True,device_map="auto")#,  is_decoder=True
# )
model = model.eval()

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
print(f"Model device: {model.device}")

config.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

configuration_falcon.py:   0%|          | 0.00/7.16k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-7b:
- configuration_falcon.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.



modeling_falcon.py:   0%|          | 0.00/56.9k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-7b:
- modeling_falcon.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


...

generation_config.json:   0%|          | 0.00/117 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.73M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/281 [00:00<?, ?B/s]

Model device: cuda:0


In [6]:
generation_config = model.generation_config
# generation_config.temperature = 0
generation_config.num_return_sequences = 1.
generation_config.max_new_tokens = 512
generation_config.use_cache = False
generation_config.repetition_penalty = 111.7
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id
# generation_config

## Stop LLM From Rambling

In [7]:
class StopGenerationCriteria(StoppingCriteria):
    def __init__(#: List[List[str]]
        self, tokens, tokenizer: AutoTokenizer, device: torch.device
    ):
        stop_token_ids = [tokenizer.convert_tokens_to_ids(t) for t in tokens]
        self.stop_token_ids = [
            torch.tensor(x, dtype=torch.long, device=device) for x in stop_token_ids
        ]

    def __call__(
        self, input_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs
    ) -> bool:
        for stop_ids in self.stop_token_ids:
            if torch.eq(input_ids[0][-len(stop_ids) :], stop_ids).all():
                return True
        return False

In [8]:
stop_tokens = [["Human", ":"], ["AI", ":"]]
stopping_criteria = StoppingCriteriaList(
    [StopGenerationCriteria(stop_tokens, tokenizer, model.device)]
)

In [9]:
generation_pipeline = pipeline(
    model=model,
    tokenizer=tokenizer,
    return_full_text=True,
    task="text-generation",
    stopping_criteria=stopping_criteria,
    generation_config=generation_config,
)

llm = HuggingFacePipeline(pipeline=generation_pipeline)
# pipeline()

The model 'FalconForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusForCausalLM', 'PLBartForCausalLM', 'ProphetNetForCausalLM', 'QDQBertLMHeadModel', 'ReformerModelWithLMHead', 'RemBertForCausalLM', 'RobertaForCausalLM', 'RobertaPreLayerNorm

In [26]:
#you are a chatbot named max and you are given with the timetable and also a message from the student  identify the kind of operation which is of the following, i.e ['reschedule','cancel','timetabel'] and return any one of the following operation and also a message to the user and return a list whose first element is the operation and the second element is the message.
#example:

#a 9-shot model request
template = """
human: can you shedule my time from 10am thursday to 5pm wednesday.
max: [operation:reschedule ,specific:10am thursday - 5pm wednesday]

human: cancel my class from 10am monday.
max: [operation:cancel ,specific:10am monday]

human: what is my time table on 9pm tuesday?
max: [operation:timetabel ,specific:9pm tuesday]

human: bro i don't wanna go tomorrow 3am.
max: [operation:cancel ,specific:3am tomorrow]

human: can you move my class from 9am thursday to 6pm monday.
max: [operation:cancel ,specific:3am tomorrow]

human: can i switch my class from 10pm to 4am from monday and thursday.
max: [operation:reschedule ,specific:10pm monday - 4am thursday]

human: i'm not attending class monday 10pm.
max: [operation:cancel ,specific:10pm monday]

human: change the shedule from 1pm sunday to 9am saturday.
max: [operation:reschedule ,specific:1pm sunday - 9am saturday]

human: can you reschedule from 8am sunday to 10pm monday.
max: [operation:reschedule ,specific:8am sunday - 10pm monday]

{history}
Human: {input}
AI:""".strip()

prompt = PromptTemplate(input_variables=["input",'history'], template=template)

# print(template)

## Conversation Chain

In [27]:
class CleanupOutputParser(BaseOutputParser):
    def parse(self, text: str) -> str:
        user_pattern = r"\nUser"
        text = re.sub(user_pattern, "", text)
        human_pattern = r"\nHuman:"
        text = re.sub(human_pattern, "", text)
        ai_pattern = r"\nAI:"
        return re.sub(ai_pattern, "", text).strip()

    @property
    def _type(self) -> str:
        return "output_parser"

In [32]:
memory = ConversationBufferWindowMemory(
    memory_key="history", k=3, return_only_outputs=True
) # memory=memory,

chain = ConversationChain(llm=llm,
                        memory=memory,
                          prompt=prompt,
                          verbose=False,
                         output_parser=CleanupOutputParser())
# dic = str(chain.predict(input="i am not attending class on 9am monday"))


In [46]:
# id = "tPAS3CycNiaMHk8DB6YvsBuClSA3"
def dele_table(id,time,day):
  collection=db.collection("timeslots").stream()
  collection1=db.collection('users').stream()
  for x in collection:
    if x.to_dict()['time']==time and x.to_dict()['day']==day:
      for t_id,s_id in zip(x.to_dict()['teachers'],x.to_dict()['students']):
        if t_id["teacherId"] == id:
          return t_id
        elif s_id["studentId"] == id:
          return s_id
      return {}
  return {}

In [34]:
def fetch_timetable():
    collection=db.collection("timeslots").stream()
    collection1=db.collection('users').stream()
    students=[]
    studentstimetable=[]
    teacherstimetable=[]
    teachers=[]
    for a in collection1:
        if a.to_dict()['role']=='Student':
            students+=[a.to_dict()['uid']]
            studentstimetable+=[{a.to_dict()['uid']:[]}]
        elif a.to_dict()['role']=='Teacher':
            teachers+=[a.to_dict()['uid']]
            teacherstimetable+=[{a.to_dict()['uid']:[]}]

    temp={}

    for i in collection:
       temp[str(i.to_dict()['day']+i.to_dict()['time'])]=(i.to_dict()['students'],i.to_dict()['teachers'])
    for i in temp.keys():
        if temp[i]==([],[]):
            temp[i]=[]
    for i in temp.keys():
        if len(temp[i])>0:
            for b in range(len(temp[i][0])):

                for c in studentstimetable:

                    try:
                        s=c[temp[i][0][b]['studentId']]
                        s+=[(temp[i][1][b]['teacherId'],temp[i][0][b]['subject'],i)]
                        studentstimetable[temp[i][0][b]['studentId']]=s
                    except:
                        pass
                for e in teacherstimetable:
                    try:
                        s=e[temp[i][1][b]['teacherId']]
                        s+=[(temp[i][0][b]['studentId'],temp[i][0][b]['subject'],i)]
                        teacherstimetable[temp[i][0][b]['teacherId']]=s
                    except:
                        pass
    yield studentstimetable,teacherstimetable



In [ ]:
def draft_email(userId,time,day,decision=None,specifics=None,data=None):
    collection1=db.collection('users').stream()
    body,subject = '',''
    for i in collection1:
        if i.to_dict()['uid']==userId:
            sname=i.to_dict()['displayName']
            to_email=i.to_dict()['email']
            role=i.to_dict()['role'].lower()
            print(sname,to_email,role)
     # Your email credentials
    if data==None:
        if decision=='reschedule' and role=='student':
            subject='Reschedule requested by the student'
            body="The student with Student name "+sname+' with student id '+userId+' has requested to reschedule the class to '+ specifics+" This is a system generated email Dont reply!!"
        elif decision=="reschedule" and role=='teacher':
            subject='Class rescheduled'
            body="The teacher "+'has requested to reschedule the class to '+ specifics+". This is a system generated email. Dont reply!!"

        elif decision=="cancel" and role=='student':
            subject="Class Cancellation request"
            body="The student with Student name "+sname+' with student id '+userId+' has requested to cancel the class '+" This is a system generated email Dont reply!!"

        elif decision=="cancel" and role=="teacher":
            subject="Class stands cancelled"
            body="The class has been cancelled by the teacher "+sname+" This is a system generated email Dont reply!!"
            for s_id_tb,t_id_tb in fetch_timetable():
              for s_id_,t_id_ in zip(t_id_tb,s_id_tb):
                print(s_id_)

    elif data==None and decision==None:
        return "Enter the correct parameters"
    else:
        subject="Alert Message !!!!"
        body=data




    sender_email = "enigmademo40@gmail.com"
    sender_password = 'lsxtqkajwahrvurq'

    # Create the MIMEText object for the email content
    msg = MIMEText(body)
    msg["Subject"] = subject
    msg["From"] = "enigmademo40@gmail.com"
    msg["To"] = to_email

    # Connect to the SMTP server (in this case, Gmail's SMTP server)
    with smtplib.SMTP("smtp.gmail.com", 587) as server:
        # Start the TLS connection
        print(server.starttls())

        # Login to your email account
        print(server.login(sender_email, sender_password))

        # Send the email
        print(server.sendmail(sender_email, to_email, msg.as_string()))
    return {"message":msg.as_string()}
    # Example usage

# draft_email("pGO5I3P89oniJyCHQopT",'cancel',"ggggg")

In [47]:

conf.get_default().auth_token = "2ZtHLeI6SwiIyMJ23MaSr2KFG49_qEfgHMUHjSNmaMLeHUXt"
print(ngrok.connect(5000))


# Create a Flask app
app = Flask(__name__)


@app.route("/",methods=['GET'])
def home():
  tag_c = request.args.get('tag')
  id = request.args.get('id')
  dic = chain.predict(input=tag_c)
  print(dic)
  op,specific = dic.strip('[').strip(']').split(',')
  op,specific = op.split(':')[-1],specific.split(':')[-1]
  print(op,specific)

  if 'reschedule' == op.lower():
    draft_email(id,op.lower(),specific)
    ca,ad = map(lambda x: x.strip(),specific.split('-'))
    time,day = ca.split()
    ttime,tday = time[:-2]+":00 "+time[-2:].upper(),day.capitalize()
    time,day = ad.split()
    ftime,fday = time[:-2]+":00 "+time[-2:].upper(),day.capitalize()
    tt = dele_table(id,ttime,tday)
    ft =  dele_table(id,ftime,fday)
    if tt!={} and ft == {}:

      return jsonify({'to':tt,'ft':ft,"message":"sceduled"})
    else:
      return jsonify({"message":"not sceduled"})

  elif 'cancel' == op.lower():
    draft_email(id,op,specific)
    time,day = specific.split()
    time,day = time[:-2]+":00 "+time[-2:].upper(),day.capitalize()
    return jsonify(dele_table(id,time,day)|{'time':time,'day':day})

  else:
    time,day = specific.split()
    time,day = time[:-2]+":00 "+time[-2:].upper(),day.capitalize()
    for s_id_tb,t_id_tb in fetch_timetable():
      for s_id_,t_id_ in zip(s_id_tb,t_id_tb):
        if s_id_.get(id,0) != 0:
          return jsonify(s_id_[id])
        elif t_id_.get(id,0) != 0:
          return jsonify(t_id_[id])
# Run the app
app.run()


Exception in thread Thread-63 (_monitor_process):
Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.10/dist-packages/pyngrok/process.py", line 140, in _monitor_process
    self._log_line(self.proc.stdout.readline())
  File "/usr/lib/python3.10/encodings/ascii.py", line 26, in decode
    return codecs.ascii_decode(input, self.errors)[0]
UnicodeDecodeError: 'ascii' codec can't decode byte 0xc2 in position 184: ordinal not in range(128)


NgrokTunnel: "https://aea4-35-227-143-171.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


[operation:cancel,specific:10pm monday]
cancel 10pm monday


INFO:werkzeug:127.0.0.1 - - [22/Dec/2023 19:24:56] "GET /?tag=i%20don't%20want%20to%20come%20to%2010pm%20class%20on%20monday&id=tPAS3CycNiaMHk8DB6YvsBuClSA3 HTTP/1.1" 200 -


[operation:cancel,specific:10pm monday]
cancel 10pm monday


INFO:werkzeug:127.0.0.1 - - [22/Dec/2023 19:28:12] "GET /?tag=i%20don't%20want%20to%20come%20to%2010pm%20class%20on%20monday&id=CsIjKHUHnkdPAO0A4DUpl2B5L4t1 HTTP/1.1" 200 -
